In [29]:
from models import *
from utils import *

In [31]:
student_img_path = "../data/student_images"
instructor_img_path = "../data/instructor_images"

prompt_path = "../prompts/csv_promt.txt"
csv_output = "output.csv"

student_response = process_form(prompt_path, student_img_path)
instructor_response = process_form(prompt_path, instructor_img_path)

In [32]:
dict_to_csv(student_response, "out.csv")

In [33]:
add_instruct_answer_to_csv(instructor_response, "out.csv")